# Hello Syft

In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
# datasets = sy.search("x")
# # Exception: You are running syft==0.8.1-beta.2 but hopeful_bach node requires 0.8.0-beta.0

In [ ]:
# asset = datasets[0].assets

In [ ]:
# ptr = asset.mock
# ptr

In [ ]:
# ptr_res = ptr.sum()
# ptr_res

In [ ]:
# @sy.syft_function(input_policy=sy.ExactMatch(data=ptr),
#                   output_policy=sy.SingleExecutionExactOutput())
# def my_function(data):
#     return data.sum()

In [ ]:
# result = ptr.domain.functions.my_function(ptr).get()
# result